In [10]:
#MSA
from Bio.Seq import Seq
import Bio.SeqIO as SeqIO
import random

sick_patient_seq = SeqIO.read('HBS.fasta', 'fasta')
print(sick_patient_seq)
print(sick_patient_seq.seq)


# number of individuals in each generation 
NUM_SEQ = 10
POP_SIZE = 10
SEQ_LEN = 15

ID: Sick
Name: Sick
Description: Sick patient HBB
Number of features: 0
Seq('ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGG...CAA', SingleLetterAlphabet())
ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGGTGCATCTGACTCCTGTGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGCTGCTGGTGGTCTACCCTTGGACCCAGAGGTTCTTTGAGTCCTTTGGGGATCTGTCCACTCCTGATGCTGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTGCACGTGGATCCTGAGAACTTCAGGCTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAAGAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAATGCCCTGGCCCACAAGTATCACTAAGCTCGCTTTCTTGCTGTCCAATTTCTATTAAAGGTTCCTTTGTTCCCTAAGTCCAACTACTAAACTGGGGGATATTATGAAGGGCCTTGAGCATCTGGATTCTGCCTAATAAAAAACATTTATTTTCATTGCAA


In [5]:
def generate_sequence(base, length=5, prob=0.5):
    pattern = ""
    a = ["A","C","G","T"]
    for i in range(0, length):
        r = random.randrange(0,4)
        p = random.random()
        #print(r)
        if p >= prob:
            pattern += base[i]
        else:
            pattern += a[r]
    return pattern 

def generate_set(base):
    seq_set = []
    for i in range(0, NUM_SEQ):
        seq_set.append(generate_sequence(base, SEQ_LEN))
       
    return seq_set

base = 'ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGG'
sequences = generate_set(base)

In [6]:
sequences

['ATGTTTGCTTCGTGA',
 'CCAGATCGTCCATAC',
 'ACTTTACATTTTGAA',
 'ACTTTTGCATCTCGG',
 'AGCTCCGCTTCGTGC',
 'ACGACGCCGTGTAGC',
 'AGGGTTGGCTCGAAA',
 'AAATTCTCTTGTAAC',
 'ACTTTTGGATCCGAC',
 'GTAGATCCTACTAAC']

In [17]:

def initial_population(sequences, population_size):
    population = []
    movements = [i for i in range(-5, 5)]
    
    for _ in range(population_size):
        population.append(random.sample(movements, len(sequences)))
        
    return population

initial_pop = initial_population(sequences, POP_SIZE)

In [18]:
initial_pop

[[-5, 1, 3, -3, 2, -1, -2, -4, 4, 0],
 [2, -1, -5, -3, 1, -4, -2, 3, 4, 0],
 [-5, 4, 3, 2, 1, -2, 0, -1, -4, -3],
 [1, 4, 0, -2, -5, 2, -3, -4, 3, -1],
 [4, -2, -5, -4, 0, -3, 2, -1, 1, 3],
 [1, -4, -1, 0, 4, -5, 3, 2, -3, -2],
 [-2, 2, 4, 1, -3, 0, -4, -1, 3, -5],
 [-4, 3, 0, -1, 1, 4, -2, -3, -5, 2],
 [3, 1, -5, 4, 0, -2, -1, -3, 2, -4],
 [-4, 3, 1, -1, 0, 2, 4, -2, -3, -5]]

In [75]:
def pad_seq(seq, pad, l ,r):
    sample_size = r - l + SEQ_LEN + 1
    sample = ['-' for i in range(sample_size)]
    
    sample[pad - l: pad - l + len(seq)] = seq

    print(sample)
    return sample

l = min(initial_pop[0])
r = max(initial_pop[0])
print(l,r)
for p,s in zip(initial_pop[0], sequences):
    pad_seq(s, p, l, r)
        

-5 4
['A', 'T', 'G', 'T', 'T', 'T', 'G', 'C', 'T', 'T', 'C', 'G', 'T', 'G', 'A', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', 'C', 'C', 'A', 'G', 'A', 'T', 'C', 'G', 'T', 'C', 'C', 'A', 'T', 'A', 'C', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '-', '-', 'A', 'C', 'T', 'T', 'T', 'A', 'C', 'A', 'T', 'T', 'T', 'T', 'G', 'A', 'A', '-', '-']
['-', '-', 'A', 'C', 'T', 'T', 'T', 'T', 'G', 'C', 'A', 'T', 'C', 'T', 'C', 'G', 'G', '-', '-', '-', '-', '-', '-', '-', '-']
['-', '-', '-', '-', '-', '-', '-', 'A', 'G', 'C', 'T', 'C', 'C', 'G', 'C', 'T', 'T', 'C', 'G', 'T', 'G', 'C', '-', '-', '-']
['-', '-', '-', '-', 'A', 'C', 'G', 'A', 'C', 'G', 'C', 'C', 'G', 'T', 'G', 'T', 'A', 'G', 'C', '-', '-', '-', '-', '-', '-']
['-', '-', '-', 'A', 'G', 'G', 'G', 'T', 'T', 'G', 'G', 'C', 'T', 'C', 'G', 'A', 'A', 'A', '-', '-', '-', '-', '-', '-', '-']
['-', 'A', 'A', 'A', 'T', 'T', 'C', 'T', 'C', 'T', 'T', 'G', 'T', 'A', 'A', 'C', '-', '-', '-', '-', '-', '-', '-

In [ ]:
def fitness_function(m=1,mm=-1,g=0, seqs):
    #gaps 0
    #mismatch -1 
    #match 1 
    #score = m + mm + g
    match, mismatch, gap = 0
    for i in range(0,len(template)):
        #match 
        if (seq[i] == template[i]):
            match += 1
            
    score = match + mismatch + gap
    
def msa(seq, seqs):
    

def initial_chromosome(initial_pop):
    